In [1]:
from language import tf_idf

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd
import numpy as np
import interview_utility as ut
from sklearn.pipeline import Pipeline
from pipeline import t_listToOneHot
import apriori

Feature engineering: 
1. steps of recipes are extracted
2. n number of columns is added according to n number of steps, the <columnsToAdd> number of columns is added
3. most frequent words from each step column are extracted
4. ratings for recipes are added


In [2]:
r_recipes = pd.read_csv('data/food-com/r_recipes.csv')
r_recipes['steps'] = r_recipes['steps'].apply(lambda x : x.replace("'", '').
                                                 replace('[', '').
                                                 replace(']','').
                                                 split(','))
pip = Pipeline([
    ("stepsToColumns", ut.ColStrColumns(columns = ['steps'], columnsToAdd = 20)), 
    ("mostFrequentWordOneHotEncoded", tf_idf.freqToOneHot())
])
output = pip.transform(r_recipes[:50])

#join information from other files
r_recipes = r_recipes[['id']].join(output, how = 'inner')
interaction_df = pd.read_csv('/home/svetik/Notebooks/food-com/interactions_train.csv')
interaction_rating_df = interaction_df.groupby(['recipe_id']).mean()['rating']
interaction_rating_df = pd.DataFrame(interaction_rating_df)
interaction_rating_df.index.rename('id')
r_r = r_recipes.join(interaction_rating_df, on = 'id', how = 'inner')
r_r.index = r_r.id
r_r = r_r.drop(columns = ['id'])
r_r['rating'] = r_r['rating'].apply(lambda x: round(x))

Most frequent words:
[['oven', 'preheat', 'preheat oven', 'degre', 'oven degre'], ['add', 'mix', 'ingredi', 'place', 'sugar'], ['mix', 'cover', 'drain', 'add', 'serv'], ['hour', 'add', 'tomato', 'boil', 'pepper'], ['add', 'garlic', 'egg', 'sugar', 'oregano'], ['egg', 'oregano', 'water', 'add', 'combin'], ['cook', 'add', 'season', 'pepper', 'ingredi'], ['stir', 'add', 'cover', 'chees', 'whiskey'], ['stir', 'simmer', 'bed', 'chile', 'cocoa'], ['add', 'hour', 'bake', 'cornmeal', 'morn'], ['bake', 'boil', 'chees', 'simmer', 'tomato'], ['toss', 'bake', 'heat', 'chees', 'turkey'], ['bake', 'chees', 'serv', 'desir', 'oil'], ['bake', 'stir', 'salt', 'pepperoni', 'rice'], ['add', 'bake', 'potato', 'mixtur', 'sauc'], ['serv', 'bake', 'overcook', 'runni', 'sugar'], ['heat', 'oil', 'freez', 'hour', 'add'], ['piec', 'serv', 'batter', 'garnish', 'return'], ['serv', 'steak', 'cook', 'stir', 'reheat serv'], ['hamburg', 'flour', 'stir blend', 'stir', 'blend'], ['cook', 'avocado', 'prego', 'deep fryer',

In [3]:
r_r = ut.dfWithZeroNullOnly(r_r.drop(columns = ['rating']))

2 values are deleted


In [4]:
r_r

,preheat,oven,sugar,add,place,mix,add_2step,drain,cover,mix_2step,...,garnish,batter,stir_18step,cook_18step,steak,stir_19step,flour,cook_20step,avocado,prego
id,,,,,,,,,,,,,,,,,,,,,
75452,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42198,0,0,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67547,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
39959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8559,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Freq_items will include the combination of items appearing more often together. The consequence can be up to 20 items

In [5]:
freq_items = apriori.detect_freq_items(r_r, min_percentage=0.01, max_len = 20)
freq_items.head(7)

Processing 27 combinations | Sampling itemset size 9


,support,itemsets
0,0.133333,(preheat)
1,0.133333,(oven)
2,0.100000,(sugar)
3,0.133333,(add)
4,0.100000,(place)
5,0.066667,(mix)
6,0.166667,(add_2step)


In [6]:
rules = apriori.get_association_rules(freq_items)


In [7]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(preheat),(oven),0.133333,0.133333,0.133333,1.0,7.5,0.115556,inf
1,(oven),(preheat),0.133333,0.133333,0.133333,1.0,7.5,0.115556,inf
2,(oil),(preheat),0.033333,0.133333,0.033333,1.0,7.5,0.028889,inf
3,(bake_13step),(preheat),0.033333,0.133333,0.033333,1.0,7.5,0.028889,inf
4,(bake_14step),(preheat),0.033333,0.133333,0.033333,1.0,7.5,0.028889,inf


Following I get the list of steps (described by frequent words ín steps). The sequence consist only from two steps.

In [12]:
seq_ = rules[(rules.support > 0.04) & (rules.lift > 1) & (rules.confidence > 0.9) & (rules.iloc[:,2] > 0.01)][['antecedents', 'consequents']]
seq_steps = apriori.allSequencesToList(seq_)


In [13]:
seq_steps

[['preheat', 'oven'],
 ['oven', 'preheat'],
 ['toss', 'place'],
 ['mix', 'add_2step'],
 ['garlic', 'simmer'],
 ['simmer', 'garlic'],
 ['garlic', 'simmer_10step'],
 ['simmer_10step', 'garlic'],
 ['add_7step', 'add_4step'],
 ['simmer_10step', 'simmer'],
 ['simmer', 'simmer_10step'],
 ['garlic', 'simmer_10step', 'simmer'],
 ['garlic', 'simmer', 'simmer_10step'],
 ['simmer_10step', 'simmer', 'garlic'],
 ['garlic', 'simmer_10step', 'simmer'],
 ['simmer_10step', 'garlic', 'simmer'],
 ['simmer', 'garlic', 'simmer_10step']]